# Mod8L5 PySpark vs. pandas (Local Parquet) — Do in Google Colab
**Goal:** See the syntax and workflow differences between **pandas** and **PySpark** using a **local Parquet** file (no cloud).  
**Format:** INSTRUCTOR → YOU DO → REFLECTION

## INSTRUCTOR — Follow Along Together (20 mins.)

### Why Parquet (local-first)
- **Columnar** → fast column scans & smaller I/O.
- **Compressed & splittable** → efficient storage & parallel reads.
- **Self-describing schema** → types are embedded in the file.
- **Predicate pushdown** (Spark) → skip non-matching row groups.

**Docs:**  
- Spark SQL/DataFrames: https://spark.apache.org/docs/latest/sql-programming-guide.html  
- Spark Reader/Writer Parquet: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrameReader.parquet.html  
- pandas `read_parquet`/`to_parquet`: https://pandas.pydata.org/docs/reference/api/pandas.read_parquet.html

### Step 0 — (Optional) Install libraries (only if needed)


### Step 1 — Create a tiny local Parquet (once) and keep it on disk
We’ll generate a small sample dataset, save it to **`./customers.parquet/`** (a folder of Parquet part files).

In [1]:
# RUN THIS CELL WITHOUT CHANGES
import pandas as pd
import numpy as np

rng = np.random.default_rng(42)
n = 30
pdf = pd.DataFrame({
    "customer_id": np.arange(1, n+1),
    "age": rng.integers(18, 70, size=n),
    "country": rng.choice(["US", "UK", "FR", "DE"], size=n),
    "spend_usd": rng.normal(100, 30, size=n).round(2)
})

# Save locally as Parquet (folder)
pdf.to_parquet("customers.parquet", index=False)
print("Wrote local parquet folder: ./customers.parquet/")


Wrote local parquet folder: ./customers.parquet/


In [2]:
pdf2 = pd.read_parquet("customers.parquet")
display(pdf2.head())
display(pdf2.dtypes)

pdf_country = (
    pdf2.groupby("country", as_index=False)
        .agg(n_rows=("customer_id","count"),
             avg_spend=("spend_usd","mean"))
        .sort_values("avg_spend", ascending=False)
)
display(pdf_country)


,customer_id,age,country,spend_usd
0,1,22,UK,164.25
1,2,58,US,87.81
2,3,52,US,84.63
3,4,40,FR,75.59
4,5,40,DE,118.48


,0
customer_id,int64
age,int64
country,object
spend_usd,float64


,country,n_rows,avg_spend
3,US,8,109.591250
0,DE,8,103.765000
2,UK,7,100.744286
1,FR,7,98.620000


### Step 3 — Start SparkSession (local) and read the same local Parquet

**Instructor:** You can hover over methods in Google Colab to see the documentation.  Note to students what is similar between this syntax and pandas syntax


In [3]:
# RUN THIS CELL WITHOUT CHANGES
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = SparkSession.builder.master("local[*]").getOrCreate()
df = spark.read.parquet("customers.parquet")
df.printSchema()
df.show(5, truncate=False)




root
 |-- customer_id: long (nullable = true)
 |-- age: long (nullable = true)
 |-- country: string (nullable = true)
 |-- spend_usd: double (nullable = true)

+-----------+---+-------+---------+
|customer_id|age|country|spend_usd|
+-----------+---+-------+---------+
|1          |22 |UK     |164.25   |
|2          |58 |US     |87.81    |
|3          |52 |US     |84.63    |
|4          |40 |FR     |75.59    |
|5          |40 |DE     |118.48   |
+-----------+---+-------+---------+
only showing top 5 rows


### Step 4 — Spark: do the same (select, filter, groupBy)


In [4]:
# Select a few columns
df_sel = df.select("customer_id", "country", "spend_usd")
df_sel.show(5, truncate=False)

# Filter
df_us = df_sel.filter(F.col("country") == "US")
print("US rows:", df_us.count())
df_us.show(truncate=False)

# Group & aggregate (Spark)
df_country = (df_sel
    .groupBy("country")
    .agg(F.count("*").alias("n_rows"),
         F.avg("spend_usd").alias("avg_spend"))
    .orderBy(F.col("avg_spend").desc())
)
df_country.show(truncate=False)


+-----------+-------+---------+
|customer_id|country|spend_usd|
+-----------+-------+---------+
|1          |UK     |164.25   |
|2          |US     |87.81    |
|3          |US     |84.63    |
|4          |FR     |75.59    |
|5          |DE     |118.48   |
+-----------+-------+---------+
only showing top 5 rows
US rows: 8
+-----------+-------+---------+
|customer_id|country|spend_usd|
+-----------+-------+---------+
|2          |US     |87.81    |
|3          |US     |84.63    |
|6          |US     |133.87   |
|11         |US     |122.3    |
|15         |US     |103.5    |
|22         |US     |118.94   |
|26         |US     |80.83    |
|28         |US     |144.85   |
+-----------+-------+---------+

+-------+------+------------------+
|country|n_rows|avg_spend         |
+-------+------+------------------+
|US     |8     |109.59125         |
|DE     |8     |103.765           |
|UK     |7     |100.74428571428571|
|FR     |7     |98.61999999999999 |
+-------+------+------------------+



### Step 5 — Write a filtered subset to Parquet & round-trip


In [5]:
out_path = "customers_high_spend.parquet"
(
    df.filter(F.col("spend_usd") > 100)
      .write.mode("overwrite")
      .parquet(out_path)
)
rt = spark.read.parquet(out_path)
rt.show(truncate=False)


+-----------+---+-------+---------+
|customer_id|age|country|spend_usd|
+-----------+---+-------+---------+
|1          |22 |UK     |164.25   |
|5          |40 |DE     |118.48   |
|6          |62 |US     |133.87   |
|10         |22 |FR     |119.52   |
|11         |45 |US     |122.3    |
|12         |68 |DE     |116.29   |
|14         |57 |UK     |106.96   |
|15         |55 |US     |103.5    |
|16         |58 |DE     |106.56   |
|17         |44 |UK     |126.14   |
|18         |24 |DE     |106.71   |
|19         |61 |FR     |120.37   |
|20         |41 |DE     |102.03   |
|21         |44 |DE     |108.67   |
|22         |37 |US     |118.94   |
|28         |60 |US     |144.85   |
|30         |41 |FR     |129.05   |
+-----------+---+-------+---------+



## YOU DO (30 mins)

Use the docs to complete the tasks in Spark **and** (where it helps you think) pandas.

**Helpful Docs:**  
- Spark `functions`: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html  
- Spark `DataFrame` API: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/index.html  
- pandas groupby: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.groupby.html

### Task A — Schema, sample, and simple transforms (Spark)
1. Print the schema and count rows.  
2. Add two columns:
   - `spend_eur = spend_usd * 0.92`
   - `is_senior = age >= 40` (boolean)
3. Show the top 8 rows sorted by `spend_usd` descending.

### Task B — Aggregations (Spark)
4. By `country`, compute:
   - `n_rows = count(*)`
   - `avg_age = avg(age)`
   - `p90_spend = percentile_approx(spend_usd, 0.90)` (use `F.expr(...)` or SQL expr)
   Sort by `p90_spend` descending, show all rows.

### Task C — (Compare) Do the same aggregation in pandas
5. Load the same `customers.parquet` into pandas and compute:
   - `n_rows`, `avg_age`, and the 90th percentile of `spend_usd` (use `quantile(0.90)`).
   - Sort to match Spark output.

> Tip: Notice Spark’s **lazy** execution vs pandas’ **eager** execution.


In [6]:
df.printSchema()
print("Rows:", df.count())

root
 |-- customer_id: long (nullable = true)
 |-- age: long (nullable = true)
 |-- country: string (nullable = true)
 |-- spend_usd: double (nullable = true)

Rows: 30


In [17]:
df_sel = df.withColumn("spend_eur", F.col("spend_usd") * F.lit(0.92)).withColumn("is_senior", F.col("age") >= F.lit(40))
df_sel.show(5, truncate=False)

+-----------+---+-------+---------+------------------+---------+
|customer_id|age|country|spend_usd|spend_eur         |is_senior|
+-----------+---+-------+---------+------------------+---------+
|1          |22 |UK     |164.25   |151.11            |false    |
|2          |58 |US     |87.81    |80.7852           |true     |
|3          |52 |US     |84.63    |77.8596           |true     |
|4          |40 |FR     |75.59    |69.5428           |true     |
|5          |40 |DE     |118.48   |109.00160000000001|true     |
+-----------+---+-------+---------+------------------+---------+
only showing top 5 rows


In [18]:
df_sel.orderBy(F.col("spend_usd").desc()).show(8, truncate=False)

+-----------+---+-------+---------+------------------+---------+
|customer_id|age|country|spend_usd|spend_eur         |is_senior|
+-----------+---+-------+---------+------------------+---------+
|1          |22 |UK     |164.25   |151.11            |false    |
|28         |60 |US     |144.85   |133.262           |true     |
|6          |62 |US     |133.87   |123.16040000000001|true     |
|30         |41 |FR     |129.05   |118.72600000000001|true     |
|17         |44 |UK     |126.14   |116.0488          |true     |
|11         |45 |US     |122.3    |112.516           |true     |
|19         |61 |FR     |120.37   |110.74040000000001|true     |
|10         |22 |FR     |119.52   |109.9584          |false    |
+-----------+---+-------+---------+------------------+---------+
only showing top 8 rows


In [19]:
df_country = df_sel.groupBy("country").agg(F.count("*").alias("n_rows"), F.avg("age").alias("avg_age"), F.percentile_approx("spend_usd", 0.90).alias("p90_spend")).orderBy(F.col("p90_spend").desc())
df_country.show(truncate=False)

+-------+------+------------------+---------+
|country|n_rows|avg_age           |p90_spend|
+-------+------+------------------+---------+
|UK     |7     |43.142857142857146|164.25   |
|US     |8     |52.5              |144.85   |
|FR     |7     |43.42857142857143 |129.05   |
|DE     |8     |43.875            |118.48   |
+-------+------+------------------+---------+



In [20]:
pdf3 = pd.read_parquet("customers.parquet")

In [21]:
pdf = pdf3.groupby("country", as_index=False).agg(n_rows=("customer_id","count"), avg_age=("age","mean"), p90_spend=("spend_usd", lambda s: s.quantile(0.90))).sort_values("p90_spend", ascending=False)

In [22]:
pdf

,country,n_rows,avg_age,p90_spend
2,UK,7,43.142857,141.384
3,US,8,52.500000,137.164
1,FR,7,43.428571,123.842
0,DE,8,43.875000,116.947


### Answers (Instructor Only)

In [2]:
# RUN THIS CELL WITHOUT CHANGES -- IMPORTANT
spark.stop()
print("Spark session stopped.")

## REFLECTION — Short Answers


1) **Why Parquet locally?**
- If you used CSV yesterday and Parquet today for the same data, what changes in performance and why?
  - Parquet is very important to store and handle big data on our local machine. It’s fast, efficient, and built for scale data, so it perform better working with large dataset. A large dataset store as a csv file will always take more time to run that a file store as a parket.

2) **Scaling Up**
- If this dataset were 100 GB on your laptop, which tool would likely still run and why?
  - If this dataset were to grow to 100 GB, I would use PySpark instead of pandas because my local machine does not have enough CPU and RAM to process that amount of data. PySpark solves this problem by splitting the large dataset across multiple virtual machines in a cluster. Each machine processes a portion of the data in parallel, and Spark then combines the results into a single output. This approach makes the analysis much faster and more efficient.

- What would you change first to handle that scale?
  - The first thing I would do is to switch the library I'm working on (Pandas to Pyspark) because pandas memory is limitted to one single machine which doesn't make it the tool to handle data at large scale. But Pyspark allows me to distrubute the work accross multilple machine working in parallele.

